In [296]:
import pyredner # pyredner will be the main Python module we import for redner.
import torch # We also import PyTorch
from typing import Optional, List
import numpy as np
from transforms3d import quaternions

In [297]:

def setPos(position, newObj=None):

    # position = torch.tensor([0.43, -0.0082,  0.0138]).float()
    
    # x ~ [0.3, 0.5]
    # y ~ [-0.1, 0.1]
    # z ~ [0.01, 0.06]
    position.requires_grad = True
    _x, _y, _z = position
    new_vertices = newObj.vertices.clone()
    new_vertices = new_vertices * 0.6400000000000001
    q = [0.9238634234206463, -0.0007149357294305102, 0.008805899762346357, -0.38262033383206623]
    R = quaternions.quat2mat(q)
    R = torch.Tensor(R)
    new_vertices = torch.matmul(new_vertices, R.T)
    new_vertices[:, 0:1] += _x
    new_vertices[:, 1:2] += _y
    new_vertices[:, 2:3] += _z
    newObj.vertices = new_vertices 

    return position, newObj




In [298]:
def render(obj_list):
    
    cam_look_at = torch.tensor([0.5, 0.0, 0.0])
    cam_position = torch.tensor([0.5, 0.0, 10.0])
    camera = pyredner.Camera(position = cam_position,
                        look_at = cam_look_at,
                        up = torch.tensor([-1.0, 0.0, 0.0]),
                        fov = torch.tensor([2.291525676350207]), # in degree
                        clip_near = 1e-2, # needs to > 0
                        resolution = (128, 128),
                        )
    scene = pyredner.Scene(camera = camera, objects = obj_list)
    chan_list = [pyredner.channels.depth]
    depth_img = pyredner.render_generic(scene, chan_list)
    # return depth_img.reshape(128,128)
    near = 0.09
    far = 0.010
    depth = near * far /(far - depth_img)
    heightmap = torch.abs(depth - torch.max(depth))
    heightmap =  heightmap*37821.71428571428 - 3407.3605408838816
    heightmap = torch.relu(heightmap)
    heightmap = torch.where(heightmap > 1.0, 6e-3, heightmap) 
    return heightmap.reshape(128,128)



In [299]:
# i = render(obj_list)
# numpy_array = i.detach().numpy()
# file_path = "_img.txt"
# np.savetxt(file_path, numpy_array)
# loss = i.sum()
# loss.backward()
# p.grad

In [300]:
# from matplotlib.pyplot import imshow
# img = i.detach()
# imshow(img.cpu())

In [301]:
import torch
import torch.nn as nn

# 定义一个简单的卷积神经网络类
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 卷积层1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # 池化层1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 卷积层2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 池化层2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 反卷积层1
        self.deconv1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        # 反卷积层2
        self.deconv2 = nn.ConvTranspose2d(32, 1, kernel_size=2, stride=2)

    def forward(self, x):
        # 前向传播
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.deconv1(x)
        x = torch.relu(x)
        x = self.deconv2(x)
        return x

testNet = SimpleCNN()


In [302]:
rendering_list = []
dir = "/Users/tingxi/BulletArm/bulletarm/pybullet/urdf/object/GraspNet1B_object/055/convex.obj"
obj = pyredner.load_obj(dir, return_objects=True)
newObj = obj[0] 
epsilon=0.002
z_epsilon=epsilon * 1e-04
position = torch.tensor([0.43, -0.0082,  0.0138], requires_grad=True).float()
xyz_position, newObj = setPos(position, newObj)
rendering_list.append(newObj)



In [303]:
tray_dir = "/Users/tingxi/Downloads/Mighty Snaget-Wolt(5)/tinker.obj"
# tray_dir = "/Users/tingxi/BulletArm/tray.obj"
t = pyredner.load_obj(tray_dir, return_objects=True)
tray = t[0]   
tray.vertices /= 1000
tray.vertices[:,0:1] +=  0.5
tray.vertices[:,1:2] += -0.0
tray.vertices[:,2:3] += -0.0
rendering_list.append(tray)

In [304]:
xyz_position = xyz_position.clone().detach()
rendering_list[0].vertices = rendering_list[0].vertices.clone().detach()
ORI_VERTICES = rendering_list[0].vertices.clone().detach()
ORI_VERTICES[:,0:1] -= xyz_position[0]
ORI_VERTICES[:,1:2] -= xyz_position[1]
ORI_VERTICES[:,2:3] -= xyz_position[2]
ORI_VERTICES.requires_grad = False

此处把这个object的原始坐标信息单独储存，便于每次渲染生成obs之后重新赋值并detach，用于下一次计算

In [308]:
for iter in range(3):
    print("iter NO. ", iter+1)
    xyz_position.requires_grad = True
    new_vertices = ORI_VERTICES.clone()
    new_vertices[:,0:1] += xyz_position[0]
    new_vertices[:,1:2] += xyz_position[1]
    new_vertices[:,2:3] += xyz_position[2]
    rendering_list[0].vertices = new_vertices.clone()

    obs = render(rendering_list)
    obs = obs.reshape(1,1,128,128)
    q_value_maps = testNet(obs)
    loss = q_value_maps.sum()
    loss.backward()
    print(xyz_position.grad)
    
    with torch.no_grad():
        x_grad, y_grad, z_grad = xyz_position.grad.clone()
        xyz_position.grad.zero_()
        x,y,z = xyz_position.clone().detach()
        # step length should be within a certain range
        x_eta = torch.clamp(x_grad.sign(), min = -epsilon,   max = epsilon).detach()
        y_eta = torch.clamp(y_grad.sign(), min = -epsilon,   max = epsilon).detach()
        z_eta = torch.clamp(z_grad.sign(), min = -z_epsilon, max = z_epsilon).detach()
        # coordinate boudary of the object, please do not change these values
        # valid range of x and y is 0.2 while for z the range is 0.000025
        # accumulated change should not exceed the boundaries
        adv_position = torch.tensor([
            torch.clamp(x + x_eta, min =  0.4, max = 0.6).detach(),
            torch.clamp(y + y_eta, min = -0.1, max = 0.1).detach(),
            torch.clamp(z + z_eta, min =  0.013800, max = 0.013825).detach()
        ])
    print("gradient: ", [x_grad, y_grad, z_grad])
    print("OG position: ", xyz_position)
    print("eta: ", [x_eta, y_eta, z_eta])
    print("ADV position: ", adv_position)
    xyz_position = adv_position.clone().detach()


iter NO.  1
tensor([-31931968.0000,  21755560.0000,     88250.4531])
gradient:  [tensor(-31931968.), tensor(21755560.), tensor(88250.4531)]
OG position:  tensor([ 0.4240, -0.0022,  0.0138], requires_grad=True)
eta:  [tensor(-0.0020), tensor(0.0020), tensor(2.0000e-07)]
ADV position:  tensor([ 4.2200e-01, -2.0000e-04,  1.3800e-02])
iter NO.  2
tensor([ 2603257.0000, -2394974.2500,  -102826.0703])
gradient:  [tensor(2603257.), tensor(-2394974.2500), tensor(-102826.0703)]
OG position:  tensor([ 4.2200e-01, -2.0000e-04,  1.3800e-02], requires_grad=True)
eta:  [tensor(0.0020), tensor(-0.0020), tensor(-2.0000e-07)]
ADV position:  tensor([ 0.4240, -0.0022,  0.0138])
iter NO.  3
tensor([-1.5615e+07,  2.1225e+07,  1.8609e+04])
gradient:  [tensor(-15615426.), tensor(21225498.), tensor(18608.6895)]
OG position:  tensor([ 0.4240, -0.0022,  0.0138], requires_grad=True)
eta:  [tensor(-0.0020), tensor(0.0020), tensor(2.0000e-07)]
ADV position:  tensor([ 4.2200e-01, -2.0000e-04,  1.3800e-02])


In [306]:
t1 = torch.tensor([1.], requires_grad=True)
t2 = torch.tensor([2.], requires_grad=False)
m = t1**2
l = m.sum()
l.backward()
t1 = t2
t1.grad

In [307]:
# test_1 = SimpleCNN()
# test_2 = SimpleCNN()
# rendering_list_2 = []
# dir_2 = "/Users/tingxi/BulletArm/bulletarm/pybullet/urdf/object/GraspNet1B_object/055/convex.obj"
# obj_2 = pyredner.load_obj(dir_2, return_objects=True)
# newObj_2 = obj_2[0] 
# position_2 = torch.tensor([0.43, -0.0082,  0.0138], requires_grad=True).float()
# xyz_position_2, newObj_2 = setPos(position_2, newObj_2)
# rendering_list_2.append(newObj_2)
# obs_2 = render(obj_list=rendering_list_2)

# for iter in range(3):
#     print("iter NO. ", iter+1)
#     q_value_maps_2 = test_1(obs_2)
#     loss_2 = q_value_maps_2.sum()
#     loss_2.backward()
#     testNet.zero_grad()
#     print(xyz_position_2.grad)
#     xyz_position_2 = xyz_position_2.detach() + torch.randn(1,1,1,1)
#     xyz_position_2.requires_grad = True
#     obs_2 = test_2(xyz_position_2)
  



